# Part III Static Replication

In [32]:
import numpy as np
import pandas as pd
import datetime  as dt
from scipy.stats import norm
from scipy import interpolate
from scipy.optimize import brentq
from scipy.integrate import quad

In [33]:
# Payoff Function

def integrand(x):
    return x **  1/3 + 1.5 * np.log(x) + 10.0


I = quad(integrand, 0.0, 1.0)
print('The integral is: %.9f' % I[0])


The integral is: 8.666666667


# Import Data

In [34]:
rates = pd.read_csv('zero_rates_20201201.csv')
spx_df = pd.read_csv('SPX_options.csv')
spy_df = pd.read_csv('SPY_options.csv')

spx_df['best_mid_price'] = (spx_df['best_bid'] + spx_df['best_offer']) / 2
spx_df['strike_price'] = spx_df['strike_price'] / 1000

spy_df['best_mid_price'] = (spy_df['best_bid'] + spy_df['best_offer']) / 2
spy_df['strike_price'] = spy_df['strike_price'] / 1000

#Interpolation interest rate
days = rates['days']
rate = rates['rate']
interpolated = interpolate.interp1d(days,rate)

## Compute Payoff For Black-Scholes (1)

In [35]:
#Pay-off function using Black Scholes Model
def black_scholes_payoff(S, K, r, sigma, T):

    first_term = (S ** (1/3) * np.exp((1 / 3) * r * T - (1 / 9) * sigma ** 2 * T))

    second_term = 1.5 * (np.log(S) + (r - 0.5 * sigma ** 2) * T)

    return np.exp(-r * T) * (first_term + second_term + 10.0)

#Pay-off function using Bachelier Model
def Bach_Payoff(S, K, r, sigma, T):
    
    def f(x):
        return (S + sigma * np.sqrt(T) * x) ** (1/3) * np.exp(-0.5 * x ** 2)
    def s(x):
        return np.log(S + sigma * np.sqrt(T) * x) * np.exp(-0.5 * x ** 2)

    bounds = float('inf')
    
    first_term = (1 / np.sqrt(2 * np.pi)) * quad(f, -bounds, bounds)[0]

    second_term = 1.5 * (1 / np.sqrt(2 * np.pi)) * quad(s, -bounds, bounds)[0]

    return (first_term + second_term + 10) * np.exp(-r * T)

# Define Models

In [36]:
#Black-Scholes Model

def BlackScholesCall(S, K, r, sigma, T):
    d1 = (np.log(S / K) + (r + sigma ** 2 / 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)

    return S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)

def BlackScholesPut(S, K, r, sigma, T):
    d1 = (np.log(S / K) + (r + sigma **2 / 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)

    return K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)

#Implied Call & Put Volatility
def bs_impliedCallVolatility(S, K, r, price, T):
    impliedVol = brentq(lambda x: price -
                        BlackScholesCall(S, K, r, x, T),
                        1e-6, 1)

    return impliedVol

def bs_impliedPutVolatility(S, K, r, price, T):
    impliedVol = brentq(lambda x: price -
                        BlackScholesPut(S, K, r, x, T),
                        1e-6, 1)
                        
    return impliedVol


# Bachelier Model
def BachelierCall(S, K, r, sigma, T):
    d = (S - K) / (S * sigma * np.sqrt(T))

    return np.exp(-r * T) * ((S-K) * norm.cdf(d) + S * sigma * np.sqrt(T) * norm.pdf(d))

def BachelierPut(S, K, r, sigma, T):
    d = (S - K) / (S * sigma * np.sqrt(T))

    return np.exp(-r * T) * ((K - S) * norm.cdf(-d) + S * sigma * np.sqrt(T) * norm.pdf(-d))

#Implied Call & Put Volatility
def bach_impliedCallVolatility(S, K, r, price, T):
    impliedVol = brentq(lambda x: price -
                        BachelierCall(S, K, r, x, T),
                        1e-6, 1)

    return impliedVol

def bach_impliedPutVolatility(S, K, r, price, T):
    impliedVol = brentq(lambda x: price -
                        BachelierPut(S, K, r, x, T),
                        1e-6, 1)
                        
    return impliedVol


In [37]:
start = dt.date(2020, 12, 1)
expiry = dt.date(2021, 1, 15)

#Calculate at SPX money sigma
T = (expiry - start).days/365
spx_S = 3662.45
spy_S = 366.02
r = interpolated(T * 365) / 100
spx_K = 3660
spy_K = 366

def calc_atm_sigma(df, T, S, r, K, dataset, model):
    # dataset = 'SPY' or 'SPX'
    # model = 'blackscholes' or 'bachelier'

    atm_call = \
        df[
            (df.strike_price >= K) & 
            (df.exdate == int(expiry.strftime('%Y%m%d'))) & 
            (df.cp_flag == 'C')
        ] \
            .best_mid_price.iloc[0, ]

    atm_put = \
        df[
            (df.strike_price >= K) & 
            (df.exdate == int(expiry.strftime('%Y%m%d'))) & 
            (df.cp_flag == 'P')
        ] \
            .best_mid_price.iloc[0, ]
    
    if model == 'blackscholes':
        sigma_call = bs_impliedCallVolatility(S, K, r, atm_call, T)
        sigma_put = bs_impliedPutVolatility(S, K, r, atm_put, T)
    else:
        sigma_call = bach_impliedCallVolatility(S, K, r, atm_call, T)
        sigma_put = bach_impliedPutVolatility(S, K, r, atm_put, T)
        
    sigma = (sigma_call + sigma_put)/2

    print(dataset + ' ATM sigma using ' + model + ': ', sigma)
    return sigma

spx_bs_sigma = calc_atm_sigma(spx_df, T, spx_S, r, spx_K, 'SPX', 'blackscholes')
spy_bs_sigma = calc_atm_sigma(spy_df, T, spy_S, r, spy_K, 'SPY', 'blackscholes')

SPX ATM sigma using blackscholes:  0.1853718842874737
SPY ATM sigma using blackscholes:  0.1848115441954425


## Compute Black-Scholes Price (1)

In [38]:
#Calculate SPX Pricing
spx_BS_Price = black_scholes_payoff(spx_S, spx_K, r, spx_bs_sigma, T)
print('SPX Black-Scholes Model Price:', spx_BS_Price)

#Calculate SPY Pricing
spy_BS_Price = black_scholes_payoff(spy_S, spy_K, r, spy_bs_sigma, T)

print('SPY Black-Scholes Model Price:', spy_BS_Price)

SPX Black-Scholes Model Price: 37.70484553986259
SPY Black-Scholes Model Price: 25.995155797750314


## Compute Bachelier Price (2)

In [39]:
bach_spx_sigma = calc_atm_sigma(spx_df, T, spx_S, r, spx_K, 'SPX', 'bachelier')
bach_spy_sigma = calc_atm_sigma(spy_df, T, spy_S, r, spy_K, 'SPY', 'bachelier')

SPX ATM sigma using bachelier:  0.1853093750058496
SPY ATM sigma using bachelier:  0.18479898144523038


In [40]:
spx_Bach_Price = Bach_Payoff(spx_S, spx_K, r, bach_spx_sigma, T)
print('SPX Bachelier Model Price:', spx_Bach_Price)

spy_Bach_Price = Bach_Payoff(spy_S, spy_K, r, bach_spy_sigma, T)
print('SPY Bachelier Model Price:', spy_Bach_Price)

SPX Bachelier Model Price: 37.71359681715785
SPY Bachelier Model Price: 26.000676619258684


### Static Replication (SABR)

In [41]:
def SABR(F, K, T, alpha, beta, rho, nu):
    X = K
    # if K is at-the-money-forward
    if abs(F - K) < 1e-12:
        numer1 = (((1 - beta) ** 2) / 24) * alpha * alpha / (F ** (2 - 2 * beta))
        numer2 = 0.25 * rho * beta * nu * alpha / (F ** (1 - beta))
        numer3 = ((2 - 3 * rho * rho) / 24) * nu * nu
        VolAtm = alpha * (1 + (numer1 + numer2 + numer3) * T) / (F ** (1 - beta))
        sabrsigma = VolAtm
    else:
        z = (nu / alpha) * ((F * X) ** (0.5 * (1 - beta))) * np.log(F / X)
        zhi = np.log((((1 - 2 * rho * z + z * z) ** 0.5) + z - rho) / (1 - rho))
        numer1 = (((1 - beta) ** 2) / 24) * ((alpha * alpha) / ((F * X) ** (1 - beta)))
        numer2 = 0.25 * rho * beta * nu * alpha / ((F * X) ** ((1 - beta) / 2))
        numer3 = ((2 - 3 * rho * rho) / 24) * nu * nu
        numer = alpha * (1 + (numer1 + numer2 + numer3) * T) * z
        denom1 = ((1 - beta) ** 2 / 24) * (np.log(F / X)) ** 2
        denom2 = (((1 - beta) ** 4) / 1920) * ((np.log(F / X)) ** 4)
        denom = ((F * X) ** ((1 - beta) / 2)) * (1 + denom1 + denom2) * zhi
        sabrsigma = numer / denom

    return sabrsigma

def SABRCall(S, K, r, alpha, beta, rho, nu, T):
    sabr_vol = SABR(S * np.exp(r * T), K, T, alpha, beta, rho, nu)
    return BlackScholesCall(S, K, r, sabr_vol, T)


def SABRPut(S, K, r, alpha, beta, rho, nu, T):
    sabr_vol = SABR(S * np.exp(r * T), K, T, alpha, beta, rho, nu)
    return BlackScholesPut(S, K, r, sabr_vol, T)


def sabrcallintegrand(K, S, r, T, alpha, beta, rho, nu):
    price = SABRCall(S, K, r, alpha, beta, rho, nu, T) * ((- 2 / 9) * K ** (-5 / 3) - 1.5 / (K ** 2))
    return price


def sabrputintegrand(K, S, r, T, alpha, beta, rho, nu):
    price = SABRPut(S, K, r, alpha, beta, rho, nu, T) * ((- 2 / 9) * K ** (-5 / 3) - 1.5 / (K ** 2))
    return price


#S = 3662.45
#r = interpolated(T * 365) / 100
#T = (expiry - start).days / 365
spx_alpha = 1.81600099
beta = 0.7
spx_rho = -0.40141522
spx_nu = 2.79103083

spy_alpha = 0.90783421
spy_rho = -0.48729513
spy_nu = 2.72885668

spx_F = spx_S * np.exp(r * T)
spy_F = spy_S * np.exp(r * T)

spx_hF = ((spx_S ** (1/3) * np.exp((1/3) * r * T)) + 1.5 * np.log(spx_S * np.exp(r * T)) + 10.0)
spy_hF = ((spy_S ** (1/3) * np.exp((1/3) * r * T)) + 1.5 * np.log(spy_S * np.exp(r * T)) + 10.0)

spx_I_put = quad(lambda x: sabrputintegrand(x, spx_S, r, T, spx_alpha, beta, spx_rho, spx_nu), 1e-6, spx_F)
spx_I_call = quad(lambda x: sabrcallintegrand(x, spx_S, r, T, spx_alpha, beta, spx_rho, spx_nu), spx_F, 5000)
spx_static_replication_price = np.exp(-r * T) * spx_hF + (spx_I_put[0] + spx_I_call[0])

spy_I_put = quad(lambda x: sabrputintegrand(x, spy_S, r, T, spy_alpha, beta, spy_rho, spy_nu), 1e-6, spy_F)
spy_I_call = quad(lambda x: sabrcallintegrand(x, spy_S, r, T, spy_alpha, beta, spy_rho, spy_nu), spy_F, 5000)
spy_static_replication_price = np.exp(-r * T) * spy_hF + (spy_I_put[0] + spy_I_call[0])

print('SPX Static Replication Price: %.9f' % spx_static_replication_price)
print('SPY Static Replication Price: %.9f' % spy_static_replication_price)

SPX Static Replication Price: 37.700414693
SPY Static Replication Price: 25.992666574


## Compute Payoff For Integrated Variance

In [42]:
# Black Scholes Model Integrated Variance Function
def BS_callintegrand(K, S, r, T, sigma):
    price = BlackScholesCall(S, K, r, sigma, T) / K**2
    return price

def BS_putintegrand(K, S, r, T, sigma):
    price = BlackScholesPut(S, K, r, sigma, T) / K**2
    return price

spx_F = spx_S * np.exp(r * T)
spx_I_put = quad(lambda x: BS_putintegrand(x, spx_S, r, T, spx_bs_sigma), 0.0, spx_F)
spx_I_call = quad(lambda x: BS_callintegrand(x, spx_S, r, T, spx_bs_sigma), spx_F, 5000)
spx_BS_E_var = 2 * np.exp(r * T) * (spx_I_put[0] + spx_I_call[0])
print('The Black Scholes expected integrated variance for SPX is: %.9f' % spx_BS_E_var)

spy_F = spy_S * np.exp(r * T)
spy_I_put = quad(lambda x: BS_putintegrand(x, spy_S, r, T, spy_bs_sigma), 0.0, spy_F)
spy_I_call = quad(lambda x: BS_callintegrand(x, spy_S, r, T, spy_bs_sigma), spy_F, 5000)
spy_BS_E_var = 2 * np.exp(r * T) * (spy_I_put[0] + spy_I_call[0])
print('The Black Scholes expected integrated variance for SPY is: %.9f' % spy_BS_E_var)

The Black Scholes expected integrated variance for SPX is: 0.004236501
The Black Scholes expected integrated variance for SPY is: 0.004210928


In [43]:
# Bachelier Model Integrated Variance Function
def Bach_callintegrand(K, S, r, T, sigma):
    price = BachelierCall(S, K, r, sigma, T) / K**2
    return price

def Bach_putintegrand(K, S, r, T, sigma):
    price = BachelierPut(S, K, r, sigma, T) / K**2
    return price


spx_F = spx_S * np.exp(r * T)
spx_I_put = quad(lambda x: Bach_putintegrand(x, spx_S, r, T, bach_spx_sigma), 0.0, spx_F)
spx_I_call = quad(lambda x: Bach_callintegrand(x, spx_S, r, T, bach_spx_sigma), spx_F, 5000)
spx_Bach_E_var = 2 * np.exp(r * T) * (spx_I_put[0] + spx_I_call[0])
print('The Bachelier expected integrated variance for SPX is: %.9f' % spx_Bach_E_var)


spy_F = spy_S * np.exp(r * T)
spy_I_put = quad(lambda x: Bach_putintegrand(x, spy_S, r, T, bach_spy_sigma), 0.0, spy_F)
spy_I_call = quad(lambda x: Bach_callintegrand(x, spy_S, r, T, bach_spy_sigma), spy_F, 5000)
spy_Bach_E_var = 2 * np.exp(r * T) * (spy_I_put[0] + spy_I_call[0])
print('The Bachelier expected integrated variance for SPY is: %.9f' % spy_Bach_E_var)

The Bachelier expected integrated variance for SPX is: 0.004260983
The Bachelier expected integrated variance for SPY is: 0.004237392


In [44]:
def SABR(F, K, T, alpha, beta, rho, nu):
    X = K
    # if K is at-the-money-forward
    if abs(F - K) < 1e-12:
        numer1 = (((1 - beta) ** 2) / 24) * alpha * alpha / (F ** (2 - 2 * beta))
        numer2 = 0.25 * rho * beta * nu * alpha / (F ** (1 - beta))
        numer3 = ((2 - 3 * rho * rho) / 24) * nu * nu
        VolAtm = alpha * (1 + (numer1 + numer2 + numer3) * T) / (F ** (1 - beta))
        sabrsigma = VolAtm
    else:
        z = (nu / alpha) * ((F * X) ** (0.5 * (1 - beta))) * np.log(F / X)
        zhi = np.log((((1 - 2 * rho * z + z * z) ** 0.5) + z - rho) / (1 - rho))
        numer1 = (((1 - beta) ** 2) / 24) * ((alpha * alpha) / ((F * X) ** (1 - beta)))
        numer2 = 0.25 * rho * beta * nu * alpha / ((F * X) ** ((1 - beta) / 2))
        numer3 = ((2 - 3 * rho * rho) / 24) * nu * nu
        numer = alpha * (1 + (numer1 + numer2 + numer3) * T) * z
        denom1 = ((1 - beta) ** 2 / 24) * (np.log(F / X)) ** 2
        denom2 = (((1 - beta) ** 4) / 1920) * ((np.log(F / X)) ** 4)
        denom = ((F * X) ** ((1 - beta) / 2)) * (1 + denom1 + denom2) * zhi
        sabrsigma = numer / denom

    return sabrsigma



def SABRCall(S, K, r, alpha, beta, rho, nu, T):
    sabr_vol = SABR(S * np.exp(r * T), K, T, alpha, beta, rho, nu)
    return BlackScholesCall(S, K, r, sabr_vol, T)


def SABRPut(S, K, r, alpha, beta, rho, nu, T):
    sabr_vol = SABR(S * np.exp(r * T), K, T, alpha, beta, rho, nu)
    return BlackScholesPut(S, K, r, sabr_vol, T)


def sabrcallintegrand(K, S, r, T, alpha, beta, rho, nu):
    price = SABRCall(S, K, r, alpha, beta, rho, nu, T) / K ** 2
    return price


def sabrputintegrand(K, S, r, T, alpha, beta, rho, nu):
    price = SABRPut(S, K, r, alpha, beta, rho, nu, T) / K ** 2
    return price

spx_alpha = 1.81600099
beta = 0.7
spx_rho = -0.40141522
spx_nu = 2.79103083

spy_alpha = 0.90783421
spy_rho = -0.48729513
spy_nu = 2.72885668

spx_F = spx_S * np.exp(r * T)
spx_I_put = quad(lambda x: sabrputintegrand(x, spx_S, r, T, spx_alpha, beta, spx_rho, spx_nu), 1e-6, spx_F)
spx_I_call = quad(lambda x: sabrcallintegrand(x, spx_S, r, T, spx_alpha, beta, spx_rho, spx_nu), spx_F, 5000)
spx_E_var = 2 * np.exp(r * T) * (spx_I_put[0] + spx_I_call[0])
print('The static replication expected integrated variance for SPX is: %.9f' % spx_E_var)

spy_F = spy_S * np.exp(r * T)
spy_I_put = quad(lambda x: sabrputintegrand(x, spy_S, r, T, spy_alpha, beta, spy_rho, spy_nu), 1e-6, spy_F)
spy_I_call = quad(lambda x: sabrcallintegrand(x, spy_S, r, T, spy_alpha, beta, spy_rho, spy_nu), spy_F, 5000)
spy_E_var = 2 * np.exp(r * T) * (spy_I_put[0] + spy_I_call[0])
print('The static replication expected integrated variance for SPY is: %.9f' % spy_E_var)

The static replication expected integrated variance for SPX is: 0.006333770
The static replication expected integrated variance for SPY is: 0.006013238


# Final Outputs

In [45]:
print('For Payoff (1)\nSPX:')
print(f'SPX Black-Scholes Model Price: {spx_BS_Price}, sigma used {spx_bs_sigma}')
print(f'SPX Bachelier Model Price: {spx_Bach_Price}, sigma used {bach_spx_sigma}')
print('SPX Static Replication Price: %.9f' % spx_static_replication_price, f', SABR params: alpha: {spx_alpha}, beta: {beta}, rho: {spx_rho}, nu: {spx_nu}')

print('\nSPY:')
print(f'SPY Black-Scholes Model Price: {spy_BS_Price}, sigma used {spy_bs_sigma}')
print(f'SPY Bachelier Model Price: {spy_Bach_Price}, sigma used {bach_spy_sigma}')
print('SPY Static Replication Price: %.9f' % spy_static_replication_price, f', SABR params: alpha: {spy_alpha}, beta: {beta}, rho: {spy_rho}, nu: {spy_nu}')

print('\nFor Payoff (2)\nSPX:')
print('The Black Scholes expected integrated variance for SPX is: %.9f' % spx_BS_E_var, f', sigma used {spx_bs_sigma}')
print('The Bachelier expected integrated variance for SPX is: %.9f' % spx_Bach_E_var, f', sigma used {bach_spx_sigma}')
print('The static replication expected integrated variance for SPX is: %.9f' % spx_E_var, f', SABR params: alpha: {spx_alpha}, beta: {beta}, rho: {spx_rho}, nu: {spx_nu}')


print('\nSPY:')
print('The Black Scholes expected integrated variance for SPY is: %.9f' % spy_BS_E_var, f', sigma used {spx_bs_sigma}')
print('The Bachelier expected integrated variance for SPY is: %.9f' % spy_Bach_E_var, f', sigma used {bach_spy_sigma}')
print('The static replication expected integrated variance for SPY is: %.9f' % spy_E_var, f', SABR params: alpha: {spy_alpha}, beta: {beta}, rho: {spy_rho}, nu: {spy_nu}')

For Payoff (1)
SPX:
SPX Black-Scholes Model Price: 37.70484553986259, sigma used 0.1853718842874737
SPX Bachelier Model Price: 37.71359681715785, sigma used 0.1853093750058496
SPX Static Replication Price: 37.700414693 , SABR params: alpha: 1.81600099, beta: 0.7, rho: -0.40141522, nu: 2.79103083

SPY:
SPY Black-Scholes Model Price: 25.995155797750314, sigma used 0.1848115441954425
SPY Bachelier Model Price: 26.000676619258684, sigma used 0.18479898144523038
SPY Static Replication Price: 25.992666574 , SABR params: alpha: 0.90783421, beta: 0.7, rho: -0.48729513, nu: 2.72885668

For Payoff (2)
SPX:
The Black Scholes expected integrated variance for SPX is: 0.004236501 , sigma used 0.1853718842874737
The Bachelier expected integrated variance for SPX is: 0.004260983 , sigma used 0.1853093750058496
The static replication expected integrated variance for SPX is: 0.006333770 , SABR params: alpha: 1.81600099, beta: 0.7, rho: -0.40141522, nu: 2.79103083

SPY:
The Black Scholes expected integra